# MCSCF

In [1]:
import veloxchem as vlx
import multipsi as mtp

## Simple MCSCF of the ozone molecule

In [2]:
mol_str = """
O 0.0000   0.0000   0.0000
O 0.0000   1.0885   0.6697
O 0.0000  -1.0885   0.6697
"""

molecule = vlx.Molecule.read_str(mol_str)
basis = vlx.MolecularBasis.read(molecule,"def2-svp")

molecule.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Before running MCSCF, we need starting orbitals. We can run a Hartree-Fock or DFT calculation using VeloxChem or simply use a cheap guess.

In [3]:
orbguess = mtp.OrbitalGuess()
start_orbs = orbguess.compute(molecule, basis)

We then need to define an active space. Here, we will use all valence 2p orbitals, i.e. 9 orbitals. This corresponds to 12 electrons. With this, we have all we need to run a MCSCF.

In [4]:
orb_space = mtp.OrbSpace(molecule, start_orbs)
orb_space.cas(12, 9)

mcscf_drv = mtp.McscfDriver()
mcscf_dict = mcscf_drv.compute(molecule, basis, orb_space)

                                                                                                                          
                          Multi-Configurational Self-Consistent Field Driver
                                                                                                                          

               Active space definition:
               ------------------------
               Number of inactive (occupied) orbitals: 6
               Number of active orbitals:              9
               Number of virtual orbitals:             27

               This is a CASSCF wavefunction: CAS(12,9)

               CI expansion:
               -------------
               Number of determinants:      7056


                                                                                                                          
               ╭────────────────────────────────────╮
               │          Driver settings           │
               ╰──────────

It is good to always check the orbitals after a calculation. The orb_space object contains the updated orbitals with the active space information.

In [5]:
viewer = mtp.OrbitalViewer()
viewer.plot(molecule, basis, orb_space)

/Users/delcey/anaconda3/envs/vlxenv/lib/python3.12/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

Dropdown(description='Orbital:', index=11, options=(('  1 occ=2.000 ene=-20.826  (alpha HOMO-11)', 0), ('  2 o…

Checkbox(value=True, description='Active')

Output()

The orbitals in the active space are sorted by occupation numbers, not by energy (although here they match). The HOMO and LUMO in the ozone molecule have occupations differing significantly from 2 and 0, highlighting that this molecule is strongly correlated and would not be well described by single reference methods like DFT.

## Orbital selection for MC-PDFT of a metal complex

The example before worked immediately as we used all valence orbitals and they were conveniently located around the HOMO-LUMO gap.

This is not always the case, and sometimes, we need manual selection. Here let's look at the metal complex chromyl chloride. We're also going to use MC-PDFT with the translated BLYP functional (tBLYP).

In [6]:
mol_str = """
Cr             0.0000        -0.0000         0.0000
O              0.9204         0.0000         1.2782
O              0.9204         0.0000        -1.2782
F             -0.9817         1.4162        -0.0000
F             -0.9817        -1.4162        -0.0000
"""
molecule = vlx.Molecule.read_str(mol_str)
basis = vlx.MolecularBasis.read(molecule, 'def2-sv(p)')
molecule.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [7]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.xcfun = "BLYP"
scf_drv.ri_coulomb = True
scf_results = scf_drv.compute(molecule, basis)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Kohn-Sham                                            
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Direct Inversion of Iterative Subspace                               
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

We ideally want the Cr - O orbitals, which form the most strongly correlated bonds, but the Cr-Cl and Cl lone pairs are close to the HOMO-LUMO gaps, so we need to manually choose them. The simplest is to use the orbital viewer from MultiPsi to select the orbitals visually and save them in a file.

In [8]:
viewer = mtp.OrbitalViewer()
viewer.plot(molecule, basis, scf_drv.molecular_orbitals)

Output()

Dropdown(description='Orbital:', index=28, options=(('  1 occ=2.000 ene=-214.810  (alpha HOMO-28)', 0), ('  2 …

Checkbox(value=False, description='Active')

Output()

In [9]:
# chosen orbitals 20 23 26 27 28 30 31 32 33 34
orb_space = mtp.OrbSpace(molecule, "input-cas.h5")

mcscf_drv = mtp.McscfDriver()
mcscf_drv.xcfun = "tBLYP"
mcscf_drv.ri = True
mcscf_drv.gradient_thresh = 1.0e-6
mcscf_drv.e_change_thresh = 1.0e-6
mcscf_drv.ci_extra_tightness = 1.0e-1
mcscf_dict = mcscf_drv.compute(molecule, basis, orb_space)

                                                                                                                          
                          Multi-Configurational Self-Consistent Field Driver
                                                                                                                          

               Active space definition:
               ------------------------
               Number of inactive (occupied) orbitals: 24
               Number of active orbitals:              10
               Number of virtual orbitals:             46

               This is a CASSCF wavefunction: CAS(10,10)

               CI expansion:
               -------------
               Number of determinants:      63504


                                                                                                                          
               ╭────────────────────────────────────╮
               │          Driver settings           │
               ╰──────

Again, we should visualize the final orbitals. Note that the occupation numbers are closer to 2 and 0 than in the ozone example, but this is mostly due to using MC-PDFT.

In [10]:
viewer = mtp.OrbitalViewer()
viewer.plot(molecule, basis, orb_space)

Output()

Dropdown(description='Orbital:', index=28, options=(('  1 occ=2.000 ene=-214.802  (alpha HOMO-28)', 0), ('  2 …

Checkbox(value=True, description='Active')

Output()

## State-averaged MCSCF for benzene
MCSCF is particularly useful to compute excited properties. There are several ways to compute excited states with MCSCF, but a very simple one is using state-averaging, i.e. computing all states simultaneously and using the same orbitals.

Here we use benzene as a simple example.

In [11]:
mol_str = """
C  1.400   0.000  0.000
C  0.700   1.212  0.000
C -0.700   1.212  0.000
C -1.400   0.000  0.000
C -0.700  -1.212  0.000
C  0.700  -1.212  0.000
H  2.480   0.000  0.000
H  1.240   2.148  0.000
H -1.240   2.148  0.000
H -2.480   0.000  0.000
H -1.240  -2.148  0.000
H  1.240  -2.148  0.000
"""
molecule = vlx.Molecule.read_str(mol_str)
basis = vlx.MolecularBasis.read(molecule, 'def2-svp')
molecule.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [12]:
orbguess = mtp.OrbitalGuess()
start_orbs = orbguess.compute(molecule, basis)

orb_space = mtp.OrbSpace(molecule, start_orbs)
orb_space.cas_orbitals([12, 17, 18, 21, 22, 23]) #found through visualization

mcscf_drv = mtp.McscfDriver()
mcscf_drv.gradient_thresh = 1.0e-6
mcscf_drv.e_change_thresh = 1.0e-6
mcscf_dict = mcscf_drv.compute(molecule, basis, orb_space, 3)

                                                                                                                          
                          Multi-Configurational Self-Consistent Field Driver
                                                                                                                          

               Active space definition:
               ------------------------
               Number of inactive (occupied) orbitals: 18
               Number of active orbitals:              6
               Number of virtual orbitals:             90

               This is a CASSCF wavefunction: CAS(6,6)

               CI expansion:
               -------------
               Number of determinants:      400


                                                                                                                          
               ╭────────────────────────────────────╮
               │          Driver settings           │
               ╰───────────

Now that we have computed all states, we can compute transition properties using a state-interaction module. By default, it computes oscillator and rotatory strengths.

In [13]:
state_interaction = mtp.StateInteraction()
si_results = state_interaction.compute(molecule, basis, mcscf_dict)

                                                                                                                          
List of oscillator strengths greather than 1e-10
                                                                                                                          
  From     to       Energy (eV)    Oscillator strength (length and velocity)
     1       2        4.87427         2.524103e-04    5.532674e-04
     1       3        8.00914         1.582334e-07    1.616711e-07
                                                                                                                          
List of rotatory strengths greather than 1e-10
                                                                                                                          
  From     to       Energy (eV)    Rot. strength (a.u. and 10^-40 cgs)
     1       2        4.87427        -4.807183e-08   -2.266316e-05
     1       3        8.00914        -7.291967e-09   -3.437751e